# ReadCSV

In [4]:
# importing required libraries
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#from google.colab import drive
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
from sklearn.ensemble import GradientBoostingClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OMKAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data_train = pd.read_csv('train.csv')
data_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
data_test = pd.read_csv('test.csv')
data_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [4]:
data_train.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
data_test.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
data_train['text'].iloc[1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [10]:
# We are going to use title as the way to check fraud or not

In [7]:
# filling NA with the blank spaces

data_train=data_train.fillna('')
data_test=data_test.fillna('')
#Didn’t 

In [8]:
data_train['text_total'] = data_train['title']+' '+data_train['author']
data_test['text_total']=data_test['title']+' '+data_test['author']

In [9]:
data_test.head()

,id,title,author,text,text_total
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...","Specter of Trump Loosens Tongues, if Not Purse..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,#NoDAPL: Native American Leaders Vow to Stay A...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...","Tim Tebow Will Attempt Another Comeback, This ..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Keiser Report: Meme Wars (E995) Truth Broadcas...


In [91]:
data_test.shape

(5200, 5)

In [10]:
data_train.head()

,id,title,author,text,label,text_total
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [13]:
#X = df.drop('label',axis=1)
y=data_train['label']
#print(X.shape)
print(y.shape)

(20800,)


In [38]:
data_test['title'].loc[1]

'Russian warships ready to strike terrorists near Aleppo'

# Data Preprocessing

# Removing of special character, stopwords removal, spaces, tab removal, making it to lower cases.

In [11]:
def preprocess_text(text):
    #preprop_text = list()
    # tqdm is for printing the status bar
    # removing punctuations
    sms_wo_punct = [x for x in text if x not in string.punctuation]
    sms_wo_punct = ''.join(sms_wo_punct)
    
    # keeping only english letters
    sms_wo_punct_only_words = [x for x in sms_wo_punct if x.isalpha() or x == " "]
    sms_wo_punct_only_words = ''.join(sms_wo_punct_only_words)
    
    # converting the data to lower case
    sms_wo_punct_only_words_lc = sms_wo_punct_only_words.lower()
    
    # removing stopwords
    sms_wo_punct_only_words_lc = sms_wo_punct_only_words_lc.split(" ")
    sms_wo_punct_only_words_lc_wo_sw = [x for x in sms_wo_punct_only_words_lc if x not in stopwords.words('english')]
    sms_wo_punct_only_words_lc_wo_sw = ' '.join(sms_wo_punct_only_words_lc_wo_sw)

    return sms_wo_punct_only_words_lc_wo_sw

In [14]:
data_processed = pd.DataFrame(columns = ["preprop_text"])
for i in tqdm(data_train["text_total"]):
    data_processed = data_processed.append({"preprop_text": preprocess_text(i)}, ignore_index = True)
data_processed

100%|███████████████████████████████████████████████████████████████████████████| 20800/20800 [01:39<00:00, 209.76it/s]


,preprop_text
0,house dem aide didnt even see comeys letter ja...
1,flynn hillary clinton big woman campus breitb...
2,truth might get fired consortiumnewscom
3,civilians killed single us airstrike identifi...
4,iranian woman jailed fictional unpublished sto...
...,...
20795,rapper ti trump poster child white supremacy j...
20796,nfl playoffs schedule matchups odds new york ...
20797,macys said receive takeover approach hudsons b...
20798,nato russia hold parallel exercises balkans al...


In [15]:
data_processed_test = pd.DataFrame(columns = ["preprop_text"])
for i in tqdm(data_test["text_total"]):
    data_processed_test = data_processed_test.append({"preprop_text": preprocess_text(i)}, ignore_index = True)
data_processed_test

100%|█████████████████████████████████████████████████████████████████████████████| 5200/5200 [00:23<00:00, 221.13it/s]


,preprop_text
0,specter trump loosens tongues purse strings si...
1,russian warships ready strike terrorists near ...
2,nodapl native american leaders vow stay winter...
3,tim tebow attempt another comeback time baseba...
4,keiser report meme wars e truth broadcast network
...,...
5195,bangladeshi traffic jam never ends new york t...
5196,john kasich signs one abortion bill ohio vetoe...
5197,california today exactly sushi new york times...
5198,us marines deployed russian border norway


In [40]:
#data_train["preprop_text"] = preprocess_text(data_train["text_total"])


In [41]:
#data_test["preprop_text"] = preprocess_text(data_test["text_total"])

In [11]:
#def decontracted(phrase):
#    phrase = re.sub(r"won't", "will not", phrase)
#    phrase = re.sub(r"can \' t", "can not", phrase)
#    phrase = re.sub(r"n \' t", " not", phrase)
#    phrase = re.sub(r" \' re", " are", phrase)
#    phrase = re.sub(r" \' s", " is", phrase)
#    phrase = re.sub(r" \' d", " would", phrase)
#    phrase = re.sub(r" \' ll", " will", phrase)
#    phrase = re.sub(r" \' t", " not", phrase)
#    phrase = re.sub(r" \' ve", " have", phrase)
#    phrase = re.sub(r" \' m", " am", phrase)
       

In [12]:
#data_train['preprop_text']=data_train['preprop_text'].apply(lambda z: decontracted(z))


In [13]:
#data_test['preprop_text']=data_test['preprop_text'].apply(lambda z: decontracted(z))

In [50]:
#len(data_train)

In [46]:
#data_test['preprop_text'][2]

# Applying Stemming instead of Lemmatization 

Applied stemming as its faster compared to Lemmatization eventhough Lemmatization gives good result but its costly for large datasets

In [17]:
from nltk.stem import PorterStemmer
ps = PorterStemmer() # creating an object of potter stemmer

# applying to the dataframe
data_processed_test["preprop_text"] = data_processed_test["preprop_text"].apply(lambda x : " ".join(ps.stem(val) for val in x.split()))

In [18]:
#ps = PorterStemmer() ## creating an object of potter stemmer

## applying to the dataframe
data_processed["preprop_text"] = data_processed["preprop_text"].apply(lambda x : " ".join(ps.stem(val) for val in x.split()))

In [31]:
#l[0]

In [97]:
data_processed_test["preprop_text"][1]

'russian warship readi strike terrorist near aleppo'

In [98]:
for d in data_processed["preprop_text"]:
    print(d)

hous dem aid didnt even see comey letter jason chaffetz tweet darrel lucu
flynn hillari clinton big woman campu breitbart daniel j flynn
truth might get fire consortiumnewscom
civilian kill singl us airstrik identifi jessica purkiss
iranian woman jail fiction unpublish stori woman stone death adulteri howard portnoy
jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart daniel nussbaum
life life luxuri elton john favorit shark pictur stare long transcontinent flight
benoît hamon win french socialist parti presidenti nomin new york time alissa j rubin
excerpt draft script donald trump qampa black church pastor new york time
backchannel plan ukrain russia courtesi trump associ new york time megan twohey scott shane
obama organ action partner soroslink indivis disrupt trump agenda aaron klein
bbc comedi sketch real housew isi caus outrag chri tomlinson
russian research discov secret nazi militari base treasur hunter arctic photo amando flavio
us 

In [61]:
l=[]
for i in data_processed["preprop_text"]:
    l.append(i)
l

['hous dem aid didnt even see comey letter jason chaffetz tweet darrel lucu',
 'flynn hillari clinton big woman campu breitbart daniel j flynn',
 'truth might get fire consortiumnewscom',
 'civilian kill singl us airstrik identifi jessica purkiss',
 'iranian woman jail fiction unpublish stori woman stone death adulteri howard portnoy',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart daniel nussbaum',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'benoît hamon win french socialist parti presidenti nomin new york time alissa j rubin',
 'excerpt draft script donald trump qampa black church pastor new york time',
 'backchannel plan ukrain russia courtesi trump associ new york time megan twohey scott shane',
 'obama organ action partner soroslink indivis disrupt trump agenda aaron klein',
 'bbc comedi sketch real housew isi caus outrag chri tomlinson',
 'russian research discov secret nazi militari 

In [65]:
m=[]
for j in data_processed_test["preprop_text"]:
    m.append(j)
m

['hous dem aid didnt even see comey letter jason chaffetz tweet darrel lucu',
 'flynn hillari clinton big woman campu breitbart daniel j flynn',
 'truth might get fire consortiumnewscom',
 'civilian kill singl us airstrik identifi jessica purkiss',
 'iranian woman jail fiction unpublish stori woman stone death adulteri howard portnoy',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart daniel nussbaum',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'benoît hamon win french socialist parti presidenti nomin new york time alissa j rubin',
 'excerpt draft script donald trump qampa black church pastor new york time',
 'backchannel plan ukrain russia courtesi trump associ new york time megan twohey scott shane',
 'obama organ action partner soroslink indivis disrupt trump agenda aaron klein',
 'bbc comedi sketch real housew isi caus outrag chri tomlinson',
 'russian research discov secret nazi militari 

In [82]:
len(m)

20800

# Supervised Learning for WordEmbedding

1) I am going to use RNN as it does not label the words while wordembedding

In [19]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding


In [20]:
from tensorflow.keras.layers import LSTM,Dense,Dropout

In [21]:
vocab_size = 5000
encoded_txt_train = [one_hot(d, vocab_size) for d in data_processed["preprop_text"]]
print(encoded_txt_train)

[[3311, 3172, 923, 3119, 1449, 4261, 2833, 224, 4090, 1002, 1615, 3477, 4888], [2044, 600, 3426, 252, 2670, 2279, 131, 4935, 2132, 2044], [4358, 206, 4706, 3267, 2711], [2361, 1578, 1500, 2793, 4800, 4020, 1678, 166], [3194, 2670, 1750, 666, 3863, 1766, 2670, 122, 1355, 4779, 3644, 4220], [1314, 3401, 2577, 1, 2114, 571, 4293, 2256, 631, 1757, 3246, 3100, 2264, 3688, 131, 4935, 166], [1618, 1618, 3422, 149, 1721, 4703, 4908, 721, 3477, 1647, 224, 439], [2602, 75, 1875, 344, 3822, 1973, 2414, 3206, 419, 4848, 611, 746, 2132, 3117], [1616, 4916, 2930, 201, 571, 4840, 1316, 379, 3180, 419, 4848, 611], [3044, 287, 18, 3912, 2740, 571, 1089, 419, 4848, 611, 511, 3134, 4994, 54], [2441, 2764, 687, 4295, 459, 3299, 1776, 571, 1162, 4031, 4261], [2250, 3239, 4934, 4498, 3591, 1288, 4260, 4923, 1545, 580], [4955, 3215, 2058, 3471, 384, 1359, 4539, 3949, 1098, 1217, 3264, 4895, 4199], [2793, 2276, 4261, 1954, 571, 3912, 4090, 572], [3962, 2147, 2516, 4171, 37, 4682, 1856, 367, 1016, 1124], [2144

In [22]:
vocab_size = 5000
encoded_txt_test = [one_hot(d, vocab_size) for d in data_processed_test["preprop_text"]]
print(encoded_txt_test)

[[1019, 571, 4009, 3515, 410, 75, 722, 61, 419, 4848, 611, 4548, 4360], [4955, 3580, 2708, 4354, 761, 4420, 2768], [2303, 1195, 3319, 3152, 611, 920, 2767, 4055, 2843, 3971, 3103, 2740], [3356, 3525, 4252, 4467, 4760, 611, 1046, 419, 4848, 611, 4935, 181], [3237, 1459, 1635, 2845, 2587, 4358, 584, 4812], [571, 2018, 1905, 1304, 3426, 4412, 766], [1363, 44, 1501, 3179, 1233, 4955, 201, 571, 131, 3893, 3242], [2773, 390, 821, 1333, 3043, 4995, 1864], [1521, 1292, 1455, 3828, 2036, 4704, 4699], [1593, 2453], [2793, 4299, 859, 3626, 571, 4994, 3168, 268], [2782, 3210, 1304, 1721, 2003, 2498, 1711], [654, 1236, 1300, 4179, 4856, 2815, 1444, 1237, 1886], [3841, 4376, 1457, 3638, 3759, 2767], [1600, 2905, 1998, 3001, 4710, 2147, 2394, 529, 1314, 571, 4210, 2018, 100, 2018, 4768, 2883], [2666, 1785, 1763, 4704, 106, 1667, 700, 419, 4848, 611, 1459, 2655], [419, 3319, 3997, 4642, 3796, 1711, 1447, 3686, 3364, 419, 4848, 611, 3858, 2939, 3929, 1170], [610, 3257, 1685, 2412, 4383, 3810, 2657, 155

In [103]:
len(data_processed_test)

5200

In [23]:
padding_train = pad_sequences(encoded_txt_train,padding='pre',maxlen=25)
padding_test = pad_sequences(encoded_txt_test,padding='pre',maxlen=25)

In [24]:
#y_train=data_train['label']
y_train=y

In [108]:
#y_test=data_test['label']

In [25]:
y_train.size

20800

In [26]:
len(padding_train)

20800

In [111]:
len(padding_test)

5200

# RNN model

I am using LSTM here can also use GRU as its useful to solve vanishing gradient descent and also as we know our RNN model have short term memory thses LSTM layers and GRU helps in rembering some important words that may be forgotten by our model

In [27]:
model = Sequential()
model.add(Embedding(vocab_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='LeakyReLU'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))


In [28]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 25, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 64)                6464      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [29]:
X_train = np.array(padding_train)
y_train_new = np.array(y_train)
X_test = np.array(padding_test)
X_train.shape,y_train_new.shape,X_test.shape

((20800, 25), (20800,), (5200, 25))

In [30]:
y_train_new

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [31]:
model.fit(X_train,y_train_new,epochs=20,batch_size=64)


Epoch 1/20
325/325 [==============================] - 7s 16ms/step - loss: 0.1411 - accuracy: 0.9403
Epoch 2/20
325/325 [==============================] - 7s 20ms/step - loss: 0.0188 - accuracy: 0.9943
Epoch 3/20
325/325 [==============================] - 7s 22ms/step - loss: 0.0071 - accuracy: 0.9980
Epoch 4/20
325/325 [==============================] - 8s 24ms/step - loss: 0.0042 - accuracy: 0.9990
Epoch 5/20
325/325 [==============================] - 7s 21ms/step - loss: 0.0036 - accuracy: 0.9990
Epoch 6/20
325/325 [==============================] - 7s 22ms/step - loss: 0.0016 - accuracy: 0.9996
Epoch 7/20
325/325 [==============================] - 7s 20ms/step - loss: 0.0019 - accuracy: 0.9995
Epoch 8/20
325/325 [==============================] - 6s 20ms/step - loss: 0.0012 - accuracy: 0.9996
Epoch 9/20
325/325 [==============================] - 6s 20ms/step - loss: 0.0014 - accuracy: 0.9997
Epoch 10/20
325/325 [==============================] - 6s 20ms/step - loss: 0.0020 - accura

In [40]:
#y_test_some=model.predict('Russian warships ready to strike terrorists near Aleppo')

In [32]:
 #= model.predict_classes(X_test)
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

In [3]:
#classes_x

In [33]:
nlp_proj = pd.DataFrame()
nlp_proj['id']=data_test['id']
nlp_proj['label'] = y_pred


In [34]:
nlp_proj

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1
...,...,...
5195,25995,0
5196,25996,0
5197,25997,0
5198,25998,1


In [2]:
#final_sub['label'].value_counts()

In [ ]:
#final_sub

In [41]:
nlp_proj.to_csv('NLP_Project_RNN.csv')

# TFIDF 
Another Vectorization Approach  